In [1]:
import pandas as pd
import geopandas as gpd
import json

In [2]:
def process_excel_with_geojson(excel_path, sheet_name, geojson_path):
    """
    Processes an Excel file to align its data with a GeoJSON file and filter matches.

    Parameters:
        excel_path (str): Path to the Excel file.
        sheet_name (str): Name of the sheet to read from the Excel file.
        geojson_path (str): Path to the GeoJSON file.

    Returns:
        tuple: A tuple containing:
            - pd.DataFrame: The processed and filtered DataFrame.
            - list: Non-matching GeoJSON countries.
    """
    # Step 1: Load GeoJSON and extract country names
    geo_data = gpd.read_file(geojson_path)
    geojson_countries = [country.strip().lower() for country in geo_data['name'].tolist()]

    # Step 2: Load the specific sheet from Excel and process starting from row 11
    df = pd.read_excel(excel_path, sheet_name=sheet_name, skiprows=10)

    # Step 3: Keep and rename relevant columns
    columns_to_keep = {
        'Region, development group, country or area of destination': 'destination',
        'Region, development group, country or area of origin': 'origin',
        1990: '1990',
        1995: '1995',
        2000: '2000',
        2005: '2005',
        2010: '2010',
        2015: '2015',
        2020: '2020',
        '1990.1': '1990.1',
        '1995.1': '1995.1',
        '2000.1': '2000.1',
        '2005.1': '2005.1',
        '2010.1': '2010.1',
        '2015.1': '2015.1',
        '2020.1': '2020.1',
        '1990.2': '1990.2',
        '1995.2': '1995.2',
        '2000.2': '2000.2',
        '2005.2': '2005.2',
        '2010.2': '2010.2',
        '2015.2': '2015.2',
        '2020.2': '2020.2',
    }
    df = df[list(columns_to_keep.keys())].rename(columns=columns_to_keep)

    # Step 4: Remove rows with NaN or 0 values in year columns
    year_columns = [
        '1990', '1995', '2000', '2005', '2010', '2015', '2020',
        '1990.1', '1995.1', '2000.1', '2005.1', '2010.1', '2015.1', '2020.1',
        '1990.2', '1995.2', '2000.2', '2005.2', '2010.2', '2015.2', '2020.2'
    ]
    df = df.dropna(subset=year_columns)
    df = df[~(df[year_columns] == 0).any(axis=1)]

    # Step 5: Align names in the DataFrame with GeoJSON using equivalences
    equivalences = {
        "   Australia*": "australia",
        "   Azerbaijan*": "azerbaijan",
        "   Bahamas": "the bahamas",
        "   Bolivia (Plurinational State of)": "bolivia",
        "   Brunei Darussalam": "brunei",
        "   China*": "china",
        "   China, Taiwan Province of China*": "taiwan",
        "   Congo": "republic of the congo",
        "   Côte d'Ivoire": "ivory coast",
        "   Cyprus*": "cyprus",
        "   Czechia": "czech republic",
        "   Dem. People's Republic of Korea": "north korea",
        "   Democratic Republic of the Congo": "democratic republic of the congo",
        "   Denmark*": "denmark",
        "   Eswatini": "swaziland",
        "   Falkland Islands (Malvinas)*": "falkland islands",
        "   Finland*": "finland",
        "   France*": "france",
        "   French Southern and Antarctic Lands": "french southern and antarctic lands",
        "   Georgia*": "georgia",
        "   Greenland*": "greenland",
        "   Guinea-Bissau": "guinea bissau",
        "   Iran (Islamic Republic of)": "iran",
        "   Kosovo": "kosovo",
        "   Lao People's Democratic Republic": "laos",
        "   Malaysia*": "malaysia",
        "   Netherlands*": "netherlands",
        "   New Caledonia*": "new caledonia",
        "   New Zealand*": "new zealand",
        "   North Macedonia": "macedonia",
        "   Northern Cyprus": "northern cyprus",
        "   Norway*": "norway",
        "   Puerto Rico*": "puerto rico",
        "   Republic of Korea": "south korea",
        "   Republic of Moldova*": "moldova",
        "   Russian Federation": "russia",
        "   Serbia*": "republic of serbia",
        "   Somalia": "somalia",
        "   Spain*": "spain",
        "   State of Palestine*": "west bank",
        "   Syrian Arab Republic": "syria",
        "   The Bahamas": "the bahamas",
        "   Timor-Leste": "east timor",
        "   Ukraine*": "ukraine",
        "   United Kingdom*": "england",
        "   United Republic of Tanzania*": "united republic of tanzania",
        "   United States of America*": "usa",
        "   Venezuela (Bolivarian Republic of)": "venezuela",
        "   Viet Nam": "vietnam",
    }


    df['destination'] = df['destination'].replace(equivalences).str.strip().str.lower()
    df['origin'] = df['origin'].replace(equivalences).str.strip().str.lower()

    # Step 6: Find non-matching GeoJSON countries before filtering
    all_countries_in_df = set(df['destination'].unique()) | set(df['origin'].unique())
    non_matching_geojson_countries = [country for country in geojson_countries if country not in all_countries_in_df]

    # Step 7: Filter rows where destination and origin match GeoJSON countries
    df_filtered = df[
        (df['destination'].isin(geojson_countries)) &
        (df['origin'].isin(geojson_countries))
    ]

    return df_filtered, non_matching_geojson_countries

# Example Usage
excel_path = 'undesa_pd_2020_ims_stock_by_sex_destination_and_origin.xlsx'
sheet_name = 'Table 1'
geojson_path = '/Users/magdalenabarros/InteractiveViz/data/world.geojson'

processed_df, non_matching_geojson = process_excel_with_geojson(excel_path, sheet_name, geojson_path)

# Display the processed DataFrame
print(processed_df)
# Get unique destinations and origins from the filtered DataFrame
unique_destinations = processed_df['destination'].unique()
unique_origins = processed_df['origin'].unique()

# Count how many unique destinations and origins exist
num_unique_destinations = len(unique_destinations)
num_unique_origins = len(unique_origins)

# Display the results
print(f"Number of unique destinations: {num_unique_destinations}")
print(f"Unique destinations: {sorted(unique_destinations)}\n")

print(f"Number of unique origins: {num_unique_origins}")
print(f"Unique origins: {sorted(unique_origins)}")


# Display non-matching GeoJSON countries
print(f"Non-matching GeoJSON countries ({len(non_matching_geojson)}): {sorted(non_matching_geojson)}")


      destination                            origin    1990    1995   2000  \
6557      burundi                           belgium     642     621    475   
6558      burundi  democratic republic of the congo   36654   41447  37951   
6559      burundi                            france     321     310    237   
6560      burundi                             kenya     186     216    202   
6561      burundi                            rwanda  221943  160197  66415   
...           ...                               ...     ...     ...    ...   
35542     vanuatu                           england     239     152     95   
35543     vanuatu                              fiji     166     146    129   
35544     vanuatu                     new caledonia     163     161    146   
35546     vanuatu                   solomon islands      83      65     51   
35547     vanuatu                  papua new guinea      50      50     50   

        2005    2010    2015    2020  1990.1  ...  2010.1  2015

In [3]:
def analyze_migration_data(processed_df):
    """
    Analyzes migration data for total migration (by origin) and immigration (by destination).

    Parameters:
        processed_df (pd.DataFrame): The filtered migration DataFrame.

    Returns:
        pd.DataFrame: Aggregated total migration (by origin) and immigration (by destination) by year.
    """
    # Step 1: Melt the DataFrame to make it long-format (Year, Value)
    year_columns = ['1990', '1995', '2000', '2005', '2010', '2015', '2020']
    melted_df = processed_df.melt(
        id_vars=['origin', 'destination'],
        value_vars=year_columns,
        var_name='Year',
        value_name='Migration'
    )

    # Step 2: Group by origin and year to calculate total migration
    total_by_origin_year = melted_df.groupby(['origin', 'Year'])['Migration'].sum().reset_index()
    total_by_origin_year = total_by_origin_year.rename(columns={'origin': 'Country', 'Migration': 'Migration'})

    # Step 3: Group by destination and year to calculate total immigration
    total_by_destination_year = melted_df.groupby(['destination', 'Year'])['Migration'].sum().reset_index()
    total_by_destination_year = total_by_destination_year.rename(columns={'destination': 'Country', 'Migration': 'Immigration'})

    # Step 4: Merge the two results on Country and Year
    combined_data = pd.merge(
        total_by_origin_year,
        total_by_destination_year,
        on=['Country', 'Year'],
        how='outer'
    ).fillna(0)  # Fill missing values with 0
    
    # Step 5: Ensure Migration and Immigration columns are integers
    combined_data['Migration'] = combined_data['Migration'].astype(int)
    combined_data['Immigration'] = combined_data['Immigration'].astype(int)


    return combined_data

# Analyze the migration data
combined_data = analyze_migration_data(processed_df)

# Display the resulting DataFrame
print(combined_data)



          Country  Year  Migration  Immigration
0     afghanistan  1990    7679357        50671
1     afghanistan  1995    4346546        55794
2     afghanistan  2000    4749865        60918
3     afghanistan  2005    4113962        67045
4     afghanistan  2010    5263310        73174
...           ...   ...        ...          ...
1192     zimbabwe  2000     351911       361947
1193     zimbabwe  2005     500684       319910
1194     zimbabwe  2010     754430       286642
1195     zimbabwe  2015    1166808       288208
1196     zimbabwe  2020    1242889       299407

[1197 rows x 4 columns]


In [4]:
def process_excel_with_geojson(excel_path, sheet_name, geojson_path):
    """
    Processes an Excel file to align its data with a GeoJSON file and filter matches.

    Parameters:
        excel_path (str): Path to the Excel file.
        sheet_name (str): Name of the sheet to read from the Excel file.
        geojson_path (str): Path to the GeoJSON file.

    Returns:
        tuple: A tuple containing:
            - pd.DataFrame: The processed and filtered DataFrame.
            - list: Non-matching GeoJSON countries.
    """
    # Step 1: Load GeoJSON and extract country names
    geo_data = gpd.read_file(geojson_path)
    geojson_countries = [country.strip().lower() for country in geo_data['name'].tolist()]

    # Step 2: Load the specific sheet from Excel and process starting from row 11
    df = pd.read_excel(excel_path, sheet_name=sheet_name, skiprows=10)

    # Step 3: Keep and rename relevant columns
    columns_to_keep = {
        'Region, development group, country or area of destination': 'destination',
        'Region, development group, country or area of origin': 'origin',
        1990: '1990',
        1995: '1995',
        2000: '2000',
        2005: '2005',
        2010: '2010',
        2015: '2015',
        2020: '2020',
        '1990.1': '1990.1',
        '1995.1': '1995.1',
        '2000.1': '2000.1',
        '2005.1': '2005.1',
        '2010.1': '2010.1',
        '2015.1': '2015.1',
        '2020.1': '2020.1',
        '1990.2': '1990.2',
        '1995.2': '1995.2',
        '2000.2': '2000.2',
        '2005.2': '2005.2',
        '2010.2': '2010.2',
        '2015.2': '2015.2',
        '2020.2': '2020.2',
    }
    df = df[list(columns_to_keep.keys())].rename(columns=columns_to_keep)

    # Step 4: Remove rows with NaN or 0 values in year columns
    year_columns = [
        '1990', '1995', '2000', '2005', '2010', '2015', '2020',
        '1990.1', '1995.1', '2000.1', '2005.1', '2010.1', '2015.1', '2020.1',
        '1990.2', '1995.2', '2000.2', '2005.2', '2010.2', '2015.2', '2020.2'
    ]
    df = df.dropna(subset=year_columns)
    df = df[~(df[year_columns] == 0).any(axis=1)]

    # Step 5: Align names in the DataFrame with GeoJSON using equivalences
    equivalences = {
        "   Australia*": "australia",
        "   Azerbaijan*": "azerbaijan",
        "   Bahamas": "the bahamas",
        "   Bolivia (Plurinational State of)": "bolivia",
        "   Brunei Darussalam": "brunei",
        "   China*": "china",
        "   China, Taiwan Province of China*": "taiwan",
        "   Congo": "republic of the congo",
        "   Côte d'Ivoire": "ivory coast",
        "   Cyprus*": "cyprus",
        "   Czechia": "czech republic",
        "   Dem. People's Republic of Korea": "north korea",
        "   Democratic Republic of the Congo": "democratic republic of the congo",
        "   Denmark*": "denmark",
        "   Eswatini": "swaziland",
        "   Falkland Islands (Malvinas)*": "falkland islands",
        "   Finland*": "finland",
        "   France*": "france",
        "   French Southern and Antarctic Lands": "french southern and antarctic lands",
        "   Georgia*": "georgia",
        "   Greenland*": "greenland",
        "   Guinea-Bissau": "guinea bissau",
        "   Iran (Islamic Republic of)": "iran",
        "   Kosovo": "kosovo",
        "   Lao People's Democratic Republic": "laos",
        "   Malaysia*": "malaysia",
        "   Netherlands*": "netherlands",
        "   New Caledonia*": "new caledonia",
        "   New Zealand*": "new zealand",
        "   North Macedonia": "macedonia",
        "   Northern Cyprus": "northern cyprus",
        "   Norway*": "norway",
        "   Puerto Rico*": "puerto rico",
        "   Republic of Korea": "south korea",
        "   Republic of Moldova*": "moldova",
        "   Russian Federation": "russia",
        "   Serbia*": "republic of serbia",
        "   Somalia": "somalia",
        "   Spain*": "spain",
        "   State of Palestine*": "west bank",
        "   Syrian Arab Republic": "syria",
        "   The Bahamas": "the bahamas",
        "   Timor-Leste": "east timor",
        "   Ukraine*": "ukraine",
        "   United Kingdom*": "england",
        "   United Republic of Tanzania*": "united republic of tanzania",
        "   United States of America*": "usa",
        "   Venezuela (Bolivarian Republic of)": "venezuela",
        "   Viet Nam": "vietnam",
    }


    df['destination'] = df['destination'].replace(equivalences).str.strip().str.lower()
    df['origin'] = df['origin'].replace(equivalences).str.strip().str.lower()

    # Step 6: Find non-matching GeoJSON countries before filtering
    all_countries_in_df = set(df['destination'].unique()) | set(df['origin'].unique())
    non_matching_geojson_countries = [country for country in geojson_countries if country not in all_countries_in_df]

    # Step 7: Filter rows where destination and origin match GeoJSON countries
    df_filtered = df[
        (df['destination'].isin(geojson_countries)) &
        (df['origin'].isin(geojson_countries))
    ]

    return df_filtered, non_matching_geojson_countries

def analyze_migration_data(processed_df):
    """
    Analyzes migration data for total migration (by origin) and immigration (by destination).

    Parameters:
        processed_df (pd.DataFrame): The filtered migration DataFrame.

    Returns:
        pd.DataFrame: Aggregated total migration (by origin) and immigration (by destination) by year.
    """
    # Step 1: Melt the DataFrame to make it long-format (Year, Value)
    year_columns = ['1990', '1995', '2000', '2005', '2010', '2015', '2020']
    melted_df = processed_df.melt(
        id_vars=['origin', 'destination'],
        value_vars=year_columns,
        var_name='Year',
        value_name='Migration'
    )

    # Step 2: Group by origin and year to calculate total migration
    total_by_origin_year = melted_df.groupby(['origin', 'Year'])['Migration'].sum().reset_index()
    total_by_origin_year = total_by_origin_year.rename(columns={'origin': 'Country', 'Migration': 'Migration'})

    # Step 3: Group by destination and year to calculate total immigration
    total_by_destination_year = melted_df.groupby(['destination', 'Year'])['Migration'].sum().reset_index()
    total_by_destination_year = total_by_destination_year.rename(columns={'destination': 'Country', 'Migration': 'Immigration'})

    # Step 4: Merge the two results on Country and Year
    combined_data = pd.merge(
        total_by_origin_year,
        total_by_destination_year,
        on=['Country', 'Year'],
        how='outer'
    ).fillna(0)  # Fill missing values with 0
    
    # Step 5: Ensure Migration and Immigration columns are integers
    combined_data['Migration'] = combined_data['Migration'].astype(int)
    combined_data['Immigration'] = combined_data['Immigration'].astype(int)


    return combined_data

def process_population_and_migration_with_geojson(
    population_path, geojson_path, combined_data
):
    """
    Processes population and migration data, matches them with a GeoJSON file,
    and outputs a JSON structure with added migration, immigration, and percentages.

    Parameters:
        population_path (str): Path to the population CSV file.
        geojson_path (str): Path to the GeoJSON file.
        combined_data (pd.DataFrame): Processed DataFrame containing migration and immigration data.

    Returns:
        list: A JSON-like list of dictionaries containing population, migration, and immigration data.
    """
    # Step 1: Load the GeoJSON file and extract IDs and names
    geo_data = gpd.read_file(geojson_path)
    geojson_df = geo_data[["name", "id"]]  # Create a DataFrame with 'name' and 'id'
    geojson_df["id"] = geojson_df["id"].str.strip().str.lower()
    geojson_df["name"] = geojson_df["name"].str.strip().str.lower()

    # Step 2: Load the population data
    population_df = pd.read_csv(population_path)

    # Step 3: Filter the years we need
    years_to_keep = ["1990", "1995", "2000", "2005", "2010", "2015", "2020"]
    population_df = population_df[population_df["Year"].astype(str).isin(years_to_keep)]

    # Step 4: Normalize the 'Code' column for matching
    population_df["Code"] = population_df["Code"].str.strip().str.lower()
    population_df["Entity"] = population_df["Entity"].str.strip().str.lower()
    population_df["Year"] = population_df["Year"].astype(str)

    # Step 5: Merge population data with GeoJSON data on 'Code' and 'id'
    merged_population = population_df.merge(
        geojson_df, left_on="Code", right_on="id", how="inner"
    )

    # Step 6: Add 'Population dots' column
    merged_population["Population dots"] = (
        merged_population["Population (historical)"] // 1_000_000
    ).astype(int)

    # Step 7: Add 'Population in millions' column
    merged_population["Population in millions"] = (
        merged_population["Population (historical)"] / 1_000_000
    ).apply(lambda x: f"{int(x):,} millions")

    # Step 8: Normalize combined_data for migration and immigration
    combined_data["Country"] = combined_data["Country"].str.strip().str.lower()
    combined_data["Year"] = combined_data["Year"].astype(str)

    # Step 9: Merge migration and immigration data with population data
    final_merged = merged_population.merge(
        combined_data,
        left_on=["name", "Year"],
        right_on=["Country", "Year"],
        how="left",
    )

    # Step 10: Convert Migration and Immigration to integers
    final_merged["Migration"] = final_merged["Migration"].fillna(0).astype(int)
    final_merged["Immigration"] = final_merged["Immigration"].fillna(0).astype(int)

    # Step 11: Add 'Migration %' and 'Immigration %' columns
    final_merged["Migration %"] = (
        (final_merged["Migration"] / final_merged["Population (historical)"] * 100)
        .round(2)
        .astype(str)
        + "%"
    )
    final_merged["Immigration %"] = (
        (final_merged["Immigration"] / final_merged["Population (historical)"] * 100)
        .round(2)
        .astype(str)
        + "%"
    )

    # Step 12: Capitalize country names and handle "USA"
    final_merged["name"] = final_merged["name"].str.title().replace("Usa", "USA")
    # Ensure Year is an integer
    final_merged["Year"] = final_merged["Year"].astype(int)

    # Step 13: Convert to JSON format
    json_result = final_merged[
        [
            "name",
            "Year",
            "Population (historical)",
            "Population dots",
            "Population in millions",
            "Migration",
            "Migration %",
            "Immigration",
            "Immigration %",
        ]
    ].rename(
        columns={
            "name": "Country",
            "Year": "Year",
            "Population (historical)": "Population",
            "Population dots": "Population dots",
            "Population in millions": "Population in millions",
            "Migration": "Migration",
            "Migration %": "Migration %",
            "Immigration": "Immigration",
            "Immigration %": "Immigration %",
        }
    ).to_dict(orient="records")

    # Save JSON to a file
    with open("population_migration_geojson.json", "w") as json_file:
        json.dump(json_result, json_file, indent=4)

    return json_result


# Example usage
excel_path = "undesa_pd_2020_ims_stock_by_sex_destination_and_origin.xlsx"
geojson_path = "/Users/magdalenabarros/InteractiveViz/data/world.geojson"
population_path = "population.csv"
sheet_name = "Table 1"

# Process migration data
processed_df, _ = process_excel_with_geojson(excel_path, sheet_name, geojson_path)
combined_data = analyze_migration_data(processed_df)

# Process population and combined migration data
result_json = process_population_and_migration_with_geojson(population_path, geojson_path, combined_data)

# Save the result
with open("population_data.json", "w") as json_file:
    json.dump(result_json, json_file, indent=4)

print(json.dumps(result_json[:5], indent=4))  # Display the first 5 records

[
    {
        "Country": "Afghanistan",
        "Year": 1990,
        "Population": 12045664,
        "Population dots": 12,
        "Population in millions": "12 millions",
        "Migration": 7679357,
        "Migration %": "63.75%",
        "Immigration": 50671,
        "Immigration %": "0.42%"
    },
    {
        "Country": "Afghanistan",
        "Year": 1995,
        "Population": 17065836,
        "Population dots": 17,
        "Population in millions": "17 millions",
        "Migration": 4346546,
        "Migration %": "25.47%",
        "Immigration": 55794,
        "Immigration %": "0.33%"
    },
    {
        "Country": "Afghanistan",
        "Year": 2000,
        "Population": 20130334,
        "Population dots": 20,
        "Population in millions": "20 millions",
        "Migration": 4749865,
        "Migration %": "23.6%",
        "Immigration": 60918,
        "Immigration %": "0.3%"
    },
    {
        "Country": "Afghanistan",
        "Year": 2005,
        "Populati

/var/folders/w7/vh3676rs0zg6dygkkjljyn5c0000gn/T/ipykernel_16684/1888652294.py:188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geojson_df["id"] = geojson_df["id"].str.strip().str.lower()
/var/folders/w7/vh3676rs0zg6dygkkjljyn5c0000gn/T/ipykernel_16684/1888652294.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geojson_df["name"] = geojson_df["name"].str.strip().str.lower()


In [5]:
def process_excel_with_geojson_to_json(excel_path, sheet_name, geojson_path):
    """
    Processes an Excel file, aligns its data with a GeoJSON file, and outputs a JSON structure
    with migration dots for each year.

    Parameters:
        excel_path (str): Path to the Excel file.
        sheet_name (str): Name of the sheet to read from the Excel file.
        geojson_path (str): Path to the GeoJSON file.

    Returns:
        tuple: A tuple containing:
            - list: JSON-like list of dictionaries with migration dots.
            - int: Number of unique countries matching GeoJSON names.
            - list: List of matched countries.
            - list: List of unmatched countries.
    """
    # Step 1: Load GeoJSON and extract country names
    geo_data = gpd.read_file(geojson_path)
    geojson_df = geo_data[['name']]  # Extract names
    geojson_df['name'] = geojson_df['name'].str.strip().str.title()  # Capitalize each word

    geojson_countries = geojson_df['name'].tolist()

    # Step 2: Load the specific sheet from Excel and process starting from row 11
    df = pd.read_excel(excel_path, sheet_name=sheet_name, skiprows=10)

    # Step 3: Keep and rename relevant columns
    columns_to_keep = {
        "Region, development group, country or area of destination": "destination",
        "Region, development group, country or area of origin": "origin",
        1990: "1990",
        1995: "1995",
        2000: "2000",
        2005: "2005",
        2010: "2010",
        2015: "2015",
        2020: "2020",
    }
    df = df[list(columns_to_keep.keys())].rename(columns=columns_to_keep)

    # Step 4: Remove rows with NaN or 0 values in year columns
    year_columns = ["1990", "1995", "2000", "2005", "2010", "2015", "2020"]
    df = df.dropna(subset=year_columns)
    df = df[~(df[year_columns] == 0).any(axis=1)]

    # Step 5: Align names in the DataFrame with GeoJSON using equivalences
    equivalences = {
        "   Australia*": "australia",
        "   Azerbaijan*": "azerbaijan",
        "   Bahamas": "the bahamas",
        "   Bolivia (Plurinational State of)": "bolivia",
        "   Brunei Darussalam": "brunei",
        "   China*": "china",
        "   China, Taiwan Province of China*": "taiwan",
        "   Congo": "republic of the congo",
        "   Côte d'Ivoire": "ivory coast",
        "   Cyprus*": "cyprus",
        "   Czechia": "czech republic",
        "   Dem. People's Republic of Korea": "north korea",
        "   Democratic Republic of the Congo": "democratic republic of the congo",
        "   Denmark*": "denmark",
        "   Eswatini": "swaziland",
        "   Falkland Islands (Malvinas)*": "falkland islands",
        "   Finland*": "finland",
        "   France*": "france",
        "   French Southern and Antarctic Lands": "french southern and antarctic lands",
        "   Georgia*": "georgia",
        "   Greenland*": "greenland",
        "   Guinea-Bissau": "guinea bissau",
        "   Iran (Islamic Republic of)": "iran",
        "   Kosovo": "kosovo",
        "   Lao People's Democratic Republic": "laos",
        "   Malaysia*": "malaysia",
        "   Netherlands*": "netherlands",
        "   New Caledonia*": "new caledonia",
        "   New Zealand*": "new zealand",
        "   North Macedonia": "macedonia",
        "   Northern Cyprus": "northern cyprus",
        "   Norway*": "norway",
        "   Puerto Rico*": "puerto rico",
        "   Republic of Korea": "south korea",
        "   Republic of Moldova*": "moldova",
        "   Russian Federation": "russia",
        "   Serbia*": "republic of serbia",
        "   Somalia": "somalia",
        "   Spain*": "spain",
        "   State of Palestine*": "west bank",
        "   Syrian Arab Republic": "syria",
        "   The Bahamas": "the bahamas",
        "   Timor-Leste": "east timor",
        "   Ukraine*": "ukraine",
        "   United Kingdom*": "england",
        "   United Republic of Tanzania*": "united republic of tanzania",
        "   United States of America*": "usa",
        "   Venezuela (Bolivarian Republic of)": "venezuela",
        "   Viet Nam": "vietnam",
    }

    df["destination"] = (
        df["destination"]
        .replace(equivalences)
        .str.strip()
        .str.title()  # Capitalize each word
    )
    df["origin"] = (
        df["origin"]
        .replace(equivalences)
        .str.strip()
        .str.title()  # Capitalize each word
    )

    # Step 6: Filter rows where destination and origin match GeoJSON countries
    df_filtered = df[
        (df["destination"].isin(geojson_countries))
        & (df["origin"].isin(geojson_countries))
    ]

    # Step 7: Create new columns for "migration dots" by dividing values by 100,000 and truncating
    for year in year_columns:
        df_filtered[f"{year} dots"] = (df_filtered[year] // 100_000).astype(int)

    # Step 8: Remove rows where all "migration dots" columns are zeros
    dot_columns = [f"{year} dots" for year in year_columns]
    mask = (df_filtered[dot_columns] != 0).any(axis=1)
    df_filtered = df_filtered[mask]

    # Step 9: Melt the DataFrame to prepare for JSON export
    melted_df = df_filtered.melt(
        id_vars=["origin", "destination"],
        value_vars=dot_columns,
        var_name="Year",
        value_name="Migration dots",
    )

    # Extract the year from the "Year dots" column
    melted_df["Year"] = melted_df["Year"].str.replace(" dots", "")

    # Step 10: Remove rows with zero "Migration dots"
    melted_df = melted_df[melted_df["Migration dots"] > 0]

    # Step 11: Convert "Usa" to "USA" only for JSON export
    melted_df["origin"] = melted_df["origin"].replace({"Usa": "USA"})
    melted_df["destination"] = melted_df["destination"].replace({"Usa": "USA"})

    # Count matched and unmatched countries
    matched_countries = set(melted_df["origin"]).intersection(set(geojson_countries))
    unmatched_countries = set(geojson_countries) - set(melted_df["origin"])

    # Step 12: Convert to JSON format
    json_result = melted_df.rename(
        columns={
            "origin": "Country",
            "destination": "Destination",
            "Year": "Year",
            "Migration dots": "Migration dots",
        }
    ).sort_values(by="Country").to_dict(orient="records")

    return json_result, len(matched_countries), list(matched_countries), list(unmatched_countries)


# Example Usage
excel_path = "undesa_pd_2020_ims_stock_by_sex_destination_and_origin.xlsx"
sheet_name = "Table 1"
geojson_path = "/Users/magdalenabarros/InteractiveViz/data/world.geojson"

# Process the Excel data with the GeoJSON and convert to JSON
result_json, matched_count, matched_countries, unmatched_countries = process_excel_with_geojson_to_json(
    excel_path, sheet_name, geojson_path
)

# Save to a JSON file
with open("migration.json", "w") as json_file:
    json.dump(result_json, json_file, indent=4)

# Display the JSON structure
print(json.dumps(result_json[:5], indent=4))  # Print the first 5 records for inspection

# Display match results
print(f"Number of unique countries matching the GeoJSON: {matched_count}")
print("Matched countries:")
print(matched_countries)
print("\nUnmatched countries:")
print(unmatched_countries)


/var/folders/w7/vh3676rs0zg6dygkkjljyn5c0000gn/T/ipykernel_16684/3002146241.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geojson_df['name'] = geojson_df['name'].str.strip().str.title()  # Capitalize each word


[
    {
        "Country": "Afghanistan",
        "Destination": "Pakistan",
        "Year": "2010",
        "Migration dots": 19
    },
    {
        "Country": "Afghanistan",
        "Destination": "Saudi Arabia",
        "Year": "2005",
        "Migration dots": 2
    },
    {
        "Country": "Afghanistan",
        "Destination": "Saudi Arabia",
        "Year": "2000",
        "Migration dots": 1
    },
    {
        "Country": "Afghanistan",
        "Destination": "Pakistan",
        "Year": "1995",
        "Migration dots": 12
    },
    {
        "Country": "Afghanistan",
        "Destination": "Turkey",
        "Year": "2020",
        "Migration dots": 1
    }
]
Number of unique countries matching the GeoJSON: 129
Matched countries:
['Albania', 'Haiti', 'Panama', 'Chile', 'Mozambique', 'Hungary', 'Uganda', 'Australia', 'Madagascar', 'South Sudan', 'Eritrea', 'Nicaragua', 'Bolivia', 'Romania', 'Germany', 'Greece', 'Cuba', 'Georgia', 'Republic Of Serbia', 'Italy', 'Jamaica', 'I

/var/folders/w7/vh3676rs0zg6dygkkjljyn5c0000gn/T/ipykernel_16684/3002146241.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[f"{year} dots"] = (df_filtered[year] // 100_000).astype(int)
/var/folders/w7/vh3676rs0zg6dygkkjljyn5c0000gn/T/ipykernel_16684/3002146241.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[f"{year} dots"] = (df_filtered[year] // 100_000).astype(int)
/var/folders/w7/vh3676rs0zg6dygkkjljyn5c0000gn/T/ipykernel_16684/3002146241.py:121: SettingWithCopyWarning